# Cartpole

## A2C Agent 

In [ ]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "CartPole-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-3

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 500
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [121]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "CartPole-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent

# When the learning rate is large, policy neural network can overflow and lead to NaNs. 
# A possible fix is to reduce lr or increase beta to lower the learning rate.

if kl_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 150
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n        
    # add randomness for better exploration
    if (avg_episode_reward <= 300) and (episode % 10 == 0):
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
    
    state_adv[0] += 0.5
    
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= 15):
        agent = DRTRPOAgent(env, gamma, lr)   
    
    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: 20.0
Episode 1: 10.5
Episode 2: 12.0
Episode 3: 11.5
Episode 4: 12.0
Episode 5: 10.5
Episode 6: 10.5
Episode 7: 14.0
Episode 8: 11.0
Episode 9: 17.5
Episode 10: 11.5
Episode 11: 18.0
Episode 12: 12.5
Episode 13: 10.5
Episode 14: 21.5
Episode 15: 30.0
Episode 16: 18.5
Episode 17: 24.5
Episode 18: 21.0
Episode 19: 18.0
Episode 20: 19.0
Episode 21: 37.5
Episode 22: 29.5
Episode 23: 45.5
Episode 24: 43.0
Episode 25: 30.0
Episode 26: 56.5
Episode 27: 55.0
Episode 28: 54.5
Episode 29: 59.0
Episode 30: 58.5
Episode 31: 72.5
Episode 32: 49.0
Episode 33: 87.0
Episode 34: 62.0
Episode 35: 59.0
Episode 36: 66.0
Episode 37: 78.0
Episode 38: 84.5
Episode 39: 83.5
Episode 40: 81.0
Episode 41: 146.0
Episode 42: 123.5
Episode 43: 88.0
Episode 44: 90.0
Episode 45: 137.0
Episode 46: 81.0
Episode 47: 235.0
Episode 48: 116.0
Episode 49: 188.0
Episode 50: 177.0
Episode 51: 249.5
Episode 52: 290.5
Episode 53: 204.5
Episode 54: 191.0
Episode 55: 198.0
Episode 56: 218.0
Episode 57: 179.5
Episode 58

In [122]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [119]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "CartPole-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 150
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    total_adv_diff += abs(state_adv[1] - state_adv[0])
    # larger beta, better stability; smaller beta, better exploration
    beta = total_adv_diff/episode 
    beta += np.random.random()*0.3-0.1
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    if (episode % 10 == 0) and (avg_episode_reward <= 350): 
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        
    state_adv[0] += 0.5
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= 15):
        agent = DRTRPOAgent(env, gamma, lr)
    
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: 20.0
Episode 1: 15.0
Episode 2: 13.5
Episode 3: 12.5
Episode 4: 11.0
Episode 5: 11.0
Episode 6: 15.5
Episode 7: 20.0
Episode 8: 20.5
Episode 9: 20.5
Episode 10: 40.0
Episode 11: 50.5
Episode 12: 86.5
Episode 13: 75.0
Episode 14: 62.5
Episode 15: 56.5
Episode 16: 46.5
Episode 17: 75.0
Episode 18: 46.0
Episode 19: 52.5
Episode 20: 50.0
Episode 21: 35.0
Episode 22: 26.0
Episode 23: 27.0
Episode 24: 17.5
Episode 25: 16.5
Episode 26: 14.5
Episode 27: 20.5
Episode 28: 18.0
Episode 29: 18.0
Episode 30: 19.0
Episode 31: 24.0
Episode 32: 19.0
Episode 33: 27.0
Episode 34: 32.0
Episode 35: 59.5
Episode 36: 71.5
Episode 37: 63.0
Episode 38: 105.0
Episode 39: 41.0
Episode 40: 33.5
Episode 41: 30.0
Episode 42: 26.5
Episode 43: 26.5
Episode 44: 19.0
Episode 45: 28.0
Episode 46: 27.0
Episode 47: 31.5
Episode 48: 38.0
Episode 49: 44.0
Episode 50: 47.0
Episode 51: 83.5
Episode 52: 142.5
Episode 53: 95.0
Episode 54: 76.5
Episode 55: 93.5
Episode 56: 53.0
Episode 57: 91.5
Episode 58: 74.5
Episo

In [120]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Sinkhorn)

In [ ]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

sink_env = "CartPole-v1"
# Create Gym environment
env = gym.make(sink_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 200
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss

    total_adv_diff += abs(state_adv[1] - state_adv[0])
    # larger beta, better stability; smaller beta, better exploration
    beta = total_adv_diff/episode 
    beta += np.random.random()*0.3-0.1
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    if (episode % 10 == 0) and (avg_episode_reward <= 350): 
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= 10):
        agent = DRTRPOAgent(env, gamma, lr)
    
    beta = 50
    policy_loss = agent.compute_policy_loss_sinkhorn(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_sink_rewards = episode_rewards
dr_trpo_sink_runtime = run_time

In [ ]:
name = './log_files/dr_trpo_sink/' + sink_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_sink_runtime, dr_trpo_sink_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')